## Object Oriented Programming - Quick and dirty Introduction

In this session, we go through a toy example of object oriented programming in python and then look at the skeleton class of a document. Clearly, this topic is vast and requires a much more through treatment than being given here. But I find it more useful to introduce you to concepts because we need them rather than because they exist. 

#### Why will we use object oriented programming in this class?
Amongst the many many reasons that OOP makes sense, for us the most important one is:

1. __Reusability__
We want to be able to write all our code for every document once and then call it again and again. Having stand alone functinons not only increases the amount of _house keeping_ but will also require you to change code at different places everytime your requirements change.

2. __Encapsulation__
Remember that you will have hundreds of documents and will have to define properties for each one of them. For example, how long is every document or what are its tokens. OOP allows you to define all of these properties at one place and then the details of these implementations can be hidden.

#### Important Terminology

1. __Class__: Is _dna_ or _blueprint_ for an object that has to be modeled. Contains attributes that contins its properties and methods that characterize its behaviors.

2. __Instance__: An individual object of a class.

3. __Instantiation__: The creation of an instance of a class.

4. __Object__: A unique instance of a class. An object comprises both data members (class variables and instance variables) and methods.

5. __Class variable__: A variable that is shared by all instances of a class. They are defined within a class but outside any of the class's methods

6. __Instance variable__: A variable that is defined inside a method and belongs only to the current instance of a class.

7. __Data member__: A class variable or instance variable that holds data associated with a class and its objects.

8. __Method__ : A function that is defined in a class definition. 

So all the chit chat aside lets get down to it 

In [1]:
import numpy as np
from nltk.tokenize import wordpunct_tokenize
from nltk import PorterStemmer

class Document():
    
    """ The Doc class rpresents a class of individul documents
    """
    
    def __init__(self, speech_year, speech_pres, speech_text):
        self.year = speech_year
        self.pres = speech_pres
        self.text = speech_text.lower()
        self.tokens = np.array(wordpunct_tokenize(self.text))
        
    def token_clean(self,length):
        """ 
        description: strip out non-alpha tokens and tokens of length > 'length'
        input: length: cut off length 
        """
        self.tokens = np.array([t for t in self.tokens if (t.isalpha() and len(t) > length)])

    def stopword_remove(self, stopwords):
        """
        description: Remove stopwords from tokens.
        input: stopwords: a suitable list of stopwords
        """
        self.tokens = np.array([t for t in self.tokens if t not in stopwords])

    def stem(self):
        """
        description: Stem tokens with Porter Stemmer.
        """
        self.tokens = np.array([PorterStemmer().stem(t) for t in self.tokens])

### Self
Notice the `self` keyword that is present all over the place in the class. `self` tells python which object it needs to work with. 

1. Therefore every method should have a `self` parameter specified.
2. Notice that when the method is called the reference to the object is passed implicitly.
3. Every data member of the class needs to be referred to using `self`. 

In [2]:
#Instantiating an object.  
speech1 = Document('1986', 'Rooster', 'this is a chicken')
print speech1

#Accessing data members
print speech1.tokens

try:
    speech1.demo_self()
except Exception as ex:
    print ex

<__main__.Document instance at 0x1084269e0>
['this' 'is' 'a' 'chicken']
Document instance has no attribute 'demo_self'


### Demo of usefulness of classes

Use this [link](http://www.codeskulptor.org/#user41_X9owzYCGupNiKYr.py) to see how useful classes can be. The link leads to a code I wrote two years ago that implements a very naive version of the game Asteroids. 

## A skeleton class structure for documents

In [4]:
import numpy as np
import codecs
import nltk
import re
from nltk.tokenize import wordpunct_tokenize
from nltk import PorterStemmer


class Corpus():
    
    """ 
    The Corpus class represents a document collection
     
    """
    def __init__(self, doc_data, stopword_file, clean_length):
        """
        Notice that the __init__ method is invoked everytime an object of the class
        is instantiated
        """
        

        #Initialise documents by invoking the appropriate class
        self.docs = [Document(doc[0], doc[1], doc[2]) for doc in doc_data] 
        
        self.N = len(self.docs)
        self.clean_length = clean_length
        
        #get a list of stopwords
        self.create_stopwords(stopword_file, clean_length)
        
        #stopword removal, token cleaning and stemming to docs
        self.clean_docs(2)
        
        #create vocabulary
        self.corpus_tokens()
        
    def clean_docs(self, length):
        """ 
        Applies stopword removal, token cleaning and stemming to docs
        """
        for doc in self.docs:
            doc.token_clean(length)
            doc.stopword_remove(self.stopwords)
            doc.stem()        
    
    def create_stopwords(self, stopword_file, length):
        """
        description: parses a file of stowords, removes words of length 'length' and 
        stems it
        input: length: cutoff length for words
               stopword_file: stopwords file to parse
        """
        
        with codecs.open(stopword_file,'r','utf-8') as f: raw = f.read()
        
        self.stopwords = (np.array([PorterStemmer().stem(word) 
                                    for word in list(raw.splitlines()) if len(word) > length]))
        
     
    def corpus_tokens(self):
        """
        description: create a set of all all tokens or in other words a vocabulary
        """
        
        #initialise an empty set
        self.token_set = set()
        for doc in self.docs:
            self.token_set = self.token_set.union(doc.tokens) 
            



In [5]:
class Document():
    
    """ The Doc class rpresents a class of individul documents
    """
    
    def __init__(self, speech_year, speech_pres, speech_text):
        self.year = speech_year
        self.pres = speech_pres
        self.text = speech_text.lower()
        self.tokens = np.array(wordpunct_tokenize(self.text))
        
    def token_clean(self,length):
        """ 
        description: strip out non-alpha tokens and tokens of length > 'length'
        input: length: cut off length 
        """
        self.tokens = np.array([t for t in self.tokens if (t.isalpha() and len(t) > length)])

    def stopword_remove(self, stopwords):
        """
        description: Remove stopwords from tokens.
        input: stopwords: a suitable list of stopwords
        """
        self.tokens = np.array([t for t in self.tokens if t not in stopwords])

    def stem(self):
        """
        description: Stem tokens with Porter Stemmer.
        """
        self.tokens = np.array([PorterStemmer().stem(t) for t in self.tokens])


We will use the presedential speech dataset to demonstrate how the class works 

In [6]:
def parse_text(textraw, regex):
    """takes raw string and performs two operations
    1. Breaks text into a list of speech, president and speech
    2. breaks speech into paragraphs
    """
    prs_yr_spch_reg = re.compile(regex, re.MULTILINE|re.DOTALL)
    
    #Each tuple contains the year, last ane of the president and the speech text
    prs_yr_spch = prs_yr_spch_reg.findall(textraw)
    
    #convert immutabe tuple to mutable list
    prs_yr_spch = [list(tup) for tup in prs_yr_spch]
    
    for i in range(len(prs_yr_spch)):
        prs_yr_spch[i][2] = prs_yr_spch[i][2].replace('\n', '')
    
    #sort
    prs_yr_spch.sort()
    
    return(prs_yr_spch)

In [7]:
text = open('./../data/pres_speech/sou_all.txt', 'r').read()
regex = "_(\d{4}).*?_[a-zA-Z]+.*?_[a-zA-Z]+.*?_([a-zA-Z]+)_\*+(\\n{2}.*?)\\n{3}"
pres_speech_list = parse_text(text, regex)

In [8]:
#Instantite the corpus class
corpus = Corpus(pres_speech_list, './../data/stopwords/stopwords.txt', 2)

print corpus
print corpus.docs[0]

<__main__.Corpus instance at 0x108448998>
<__main__.Document instance at 0x1084489e0>


In [9]:
doc1 = corpus.docs[0]

In [10]:
print 'Document class data members'
print doc1.pres
print doc1.year
print doc1.tokens[0:5]
print doc1.text[0:100]

Document class data members
Washington
1790
[u'fellow' u'citizen' u'senat' u'hous' u'repres']
 fellow-citizens of the senate and house of representatives: in meeting you again i feel much satisf


In [ ]:
dimD = len(corpus.docs)
dimV = len(corpus.token_set)
# record arrays are not our friends
# so we use a dict to store term - document term matrix indices
terms_arr = [str(term) for term in corpus.token_set]
#print(terms_arr)
arr = np.zeros((dimD, dimV))
document_term_matrix = np.array(arr)
print document_term_matrix.shape

for i in range(len(corpus.docs)):
    document = corpus.docs[i]
    print 'counting terms for doc: ' + str(i)
    for token in document.tokens:
        # get document term index for the word
        term_idx = terms_arr.index(token)
        doc_term_tuple = (i, term_idx)
        document_term_matrix.itemset(doc_term_tuple, document_term_matrix.item(doc_term_tuple) + 1)

(236, 13568)
counting terms for doc: 0
counting terms for doc: 1
counting terms for doc: 2
counting terms for doc: 3
counting terms for doc: 4
counting terms for doc: 5
counting terms for doc: 6
counting terms for doc: 7
counting terms for doc: 8
counting terms for doc: 9
counting terms for doc: 10
counting terms for doc: 11
counting terms for doc: 12
counting terms for doc: 13
counting terms for doc: 14
counting terms for doc: 15
counting terms for doc: 16
counting terms for doc: 17
counting terms for doc: 18
counting terms for doc: 19
counting terms for doc: 20
counting terms for doc: 21
counting terms for doc: 22
counting terms for doc: 23
counting terms for doc: 24
counting terms for doc: 25
counting terms for doc: 26
counting terms for doc: 27
counting terms for doc: 28
counting terms for doc: 29
counting terms for doc: 30
counting terms for doc: 31
counting terms for doc: 32
counting terms for doc: 33
counting terms for doc: 34
counting terms for doc: 35
counting terms for doc: 3

In [ ]:
# HW 1, pt 1
# document_term_matrix - which returns a D by V array of frequency counts.
# will go in the corpus class which already has such things.
def document_term_matrix(corpus):
    # initiate an empty list to store dictionaries for each term
    # magnitude of V
    dimV = len(corpus.token_set)
    dimD = len(corpus.docs)
    # Assumption: terms == tokens, not clear to me if this is the case
    document_term_matrix = np.array([[0]*dimD*]dimV, dtype = [(term, 'a25') for term in corpus.corpus_tokens])
    for i in range(len(documents)):
        document = self.documents[i]
        
        for word in range(len(document.words)):
            
    